# Импорт библиотек  

Устанавливаем предпочтительную кодировку UTF-8

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Unknown instance spec: Please select VM configuration

In [8]:
%cd /home/jupyter/datasphere/project/clip

/home/jupyter/work/resources/clip


In [ ]:
!git clone --recursive https://github.com/camenduru/gaussian-splatting

In [2]:
%pip install plyfile

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
%cd /home/jupyter/datasphere/project/clip/gaussian-splatting

# Очистка старых сборок
!rm -rf submodules/diff-gaussian-rasterization/build
!rm -rf submodules/simple-knn/build

# Установка с правильной архитектурой
!TORCH_CUDA_ARCH_LIST="7.0"
%pip install submodules/simple-knn/
!TORCH_CUDA_ARCH_LIST="7.0"
%pip install submodules/diff-gaussian-rasterization/

Unknown instance spec: Please select VM configuration

In [ ]:
%pip install -q vtk


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
%pip install git+https://github.com/facebookresearch/ImageBind

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/ImageBind to /tmp/pip-req-build-6o9e8mio
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind /tmp/pip-req-build-6o9e8mio
  Resolved https://github.com/facebookresearch/ImageBind to commit 3fcf5c9039de97f6ff5528ee4a9dce903c5979b3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-yt8_wfwq/pytorchvideo_e8910367c2d241a0a5ec7fb52236f94a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-yt8_wfwq/pytorchvideo_e8910367c2d241a0a5ec7fb52236f94a
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d2

In [11]:
%pip install -q open_clip_torch celluloid matplotlib tqdm

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
%cd /home/jupyter/datasphere/project/clip/gaussian-splatting

# Импорт стандартных и внешних библиотек
import os
import sys
import uuid
from random import randint
from argparse import ArgumentParser, Namespace
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm  # для отображения прогресса

# Импорт из PyTorch
import torch

# Импорт утилит проекта
from utils.loss_utils import l1_loss, ssim  # функции для вычисления потерь
from utils.sh_utils import SH2RGB           # преобразование сферических гармоник в RGB
from utils.general_utils import safe_state  # сохранение состояния модели
from utils.image_utils import psnr          # вычисление PSNR (качество изображения)

# Импорт рендерера и интерфейса
from gaussian_renderer import render, network_gui

# Импорт компонентов сцены
from scene import Scene, GaussianModel
from scene.cameras import Camera
from scene.gaussian_model import BasicPointCloud

# Импорт параметров модели, оптимизации и пайплайна
from arguments import ModelParams, PipelineParams, OptimizationParams

# Импорт библиотеки для работы с изображениями
from PIL import Image

# Импорт OpenCLIP — модели для сопоставления текста и изображений
import open_clip

from torchvision.transforms import Normalize

/home/jupyter/work/resources/clip/gaussian-splatting


# Загрузка предобученной модели

Модель CLIP используется для оценки соответствия между текстовым описанием и сгенерированным изображением, выступая в роли дифференцируемого критика (loss-функции) в процессе оптимизации параметров гауссиан.

In [ ]:
oclip = 'ViT-B-32'
model, _, preprocess = open_clip.create_model_and_transforms(oclip, pretrained='laion2b_s34b_b79k')

model.requires_grad_(False).cuda().half()
tokenizer = open_clip.get_tokenizer(oclip)

/home/jupyter/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Перед подачей изображений в модель CLIP необходимо привести их к ожидаемому формату с помощью нормализации

In [ ]:

clippp = Normalize(mean=model.visual.image_mean, std=model.visual.image_std)
clippp

Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))

In [6]:
model = torch.jit.script(model)

В этом фрагменте создаётся синтетическое облако точек — полезное для тестирования и визуализации в сценах, не требующих реальных данных

In [ ]:
num_pts = 100
print(f"Generating random point cloud ({num_pts})...")

# Пространственные границы, в пределах которых будут размещены точки
camera_extent = 5.0

# Генерация случайных координат (x, y, z) с нормальным распределением
xyz = np.random.normal(size=(num_pts, 3))

# Масштабирование и нормализация — точки размещаются на сфере радиуса ≈1.3
xyz = 1.3 * xyz / np.linalg.norm(xyz, axis=-1, keepdims=True)

# Случайные цвета для точек, заданные в виде сферических гармоник (SH)
shs = np.random.random((num_pts, 3)) / 255.0

# Создание объекта облака точек без нормалей
pcd = BasicPointCloud(
    points=xyz,                # координаты точек
    colors=SH2RGB(shs),        # преобразование SH → RGB
    normals=np.zeros((num_pts, 3))  # нормали пока не используются
)


Generating random point cloud (100)...


Эта функция создаёт несколько случайных кропов с масштабированием и трансформацией, полезных для обучения моделей с аугментацией.

In [8]:
def random_crops(img, n, resize_to=224, min_size=0.5, max_size=1.0):
    sizes = torch.rand(n) * (max_size - min_size) + min_size
    top_left = torch.rand(n, 2) * (1.0 - sizes[:, None])
    transforms = torch.eye(2).unsqueeze(0).repeat(n, 1, 1) * sizes[:, None, None]
    transforms = torch.cat([transforms, top_left[:, :, None]], dim=-1)
    grids = torch.nn.functional.affine_grid(transforms, [n, img.shape[1], resize_to, resize_to], align_corners=True).to(img.device)
    return torch.nn.functional.grid_sample(img.repeat(n, 1, 1, 1), grids, align_corners=True)


# Функция training

Этот блок отвечает за генерацию случайного ракурса камеры, рендеринг сцены с текущими параметрами гауссиан, а затем сравнение полученного изображения с текстовым описанием с помощью модели CLIP.

Основные шаги:

- Генерируется случайная позиция камеры вокруг объекта и формируется матрица поворота и сдвига (`look_at`).
- Создается объект камеры с заданными параметрами обзора и разрешением.
- Выполняется рендеринг изображения сцены из заданного ракурса.
- Текстовый запрос форматируется с учетом выбранного ракурса.
- С помощью CLIP происходит кодирование изображения (с применением случайных обрезок) и текста в векторные представления.
- Вычисляется косинусное сходство между этими векторами, формируя функцию потерь.
- Потеря используется для обратного распространения ошибки и оптимизации параметров модели.


In [9]:
from celluloid import Camera as PltCamera
from math import floor
try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_FOUND = True
except ImportError:
    TENSORBOARD_FOUND = False

def rot2d(angle):
    return torch.tensor([
        [torch.cos(angle), torch.sin(angle), torch.tensor(0)],
       [-torch.sin(angle), torch.cos(angle), torch.tensor(0)],
        [torch.tensor(0), torch.tensor(0), torch.tensor(1)]
    ])

def normalize(x):
    return x / x.norm(dim=-1, keepdim=True)
def look_at(eye, at, up):
    z = normalize(eye - at)
    x = normalize(torch.linalg.cross(up, z))
    y = normalize(torch.linalg.cross(z, x))
    rot = torch.stack([x, y, z]).T
    trans = torch.tensor([x @ eye, y @ eye, z @ eye])
    return rot, trans


def show(tensor):
    tensor = tensor.detach().permute(1, 2, 0)
    plt.imshow(tensor.cpu().numpy())

def training(prompt, dataset, opt, pipe, testing_iterations, saving_iterations, checkpoint_iterations, checkpoint, debug_from):
    first_iter = 0
    gaussians = GaussianModel(dataset.sh_degree)
    # scene = Scene(dataset, gaussians)
    print(gaussians)
    gaussians.create_from_pcd(pcd, camera_extent)
    gaussians.training_setup(opt)
    if checkpoint:
        (model_params, first_iter) = torch.load(checkpoint)
        gaussians.restore(model_params, opt)

    bg_color = [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

    iter_start = torch.cuda.Event(enable_timing = True)
    iter_end = torch.cuda.Event(enable_timing = True)

    viewpoint_stack = None
    ema_loss_for_log = 0.0
    progress_bar = tqdm(range(first_iter, opt.iterations), desc="Training progress")
    first_iter += 1
    # dbgfig = plt.figure()
    fig = plt.figure()
    camera = PltCamera(fig)

    for iteration in range(first_iter, opt.iterations + 1):

        iter_start.record()

        gaussians.update_learning_rate(iteration)

        # Every 1000 its we increase the levels of SH up to a maximum degree
        if iteration % 1000 == 0:
            gaussians.oneupSHdegree()

        # Pick a random Camera
        # if not viewpoint_stack:
        #    viewpoint_stack = scene.getTrainCameras().copy()
        # viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack)-1))
        rand_angle = torch.rand(1) * 3.14159 * 2.0
        rand_pos = camera_extent * torch.tensor([rand_angle.cos(), 0.0, rand_angle.sin()])
        rand_pos[1] = 0.2 + (torch.rand(1) * 2 - 1) * 0.3
        dist, direc = torch.linalg.norm(rand_pos), normalize(rand_pos)
        rand_pos = direc * camera_extent


        # if (iteration % 10) == 0:
        #     rand_pos = torch.tensor([0.0, 0.0, camera_extent])
        #     rand_pos = rand_pos + torch.tensor([torch.tensor(3.14159 * 4.0 * iteration / opt.iterations).cos(), torch.tensor(3.14159 * 4.0 * iteration / opt.iterations).sin(), 0.0])

        rot, trans = look_at(rand_pos, torch.tensor([0.0, 0.0, 0.0]), torch.tensor([0.0, 1.0, 0.0]))

        train_res = floor(512 * (iteration / opt.iterations))
        train_res = 32 * (1 + train_res // 32)
        viewpoint_cam = Camera(
            colmap_id=0,
            R=rot.numpy(), #rand_angle.numpy(),
            T=trans.numpy(), #torch.zeros((3,)).numpy(),
            FoVx=90.0,
            FoVy=90.0,
            image=torch.zeros((3, train_res, train_res)),
            gt_alpha_mask=torch.ones((train_res, train_res)),
            image_name="test",
            uid=0,
        )
        # Render
        if (iteration - 1) == debug_from:
            pipe.debug = True
        render_pkg = render(viewpoint_cam, gaussians, pipe, bg_color=torch.rand(3).cuda())
        image, viewspace_point_tensor, visibility_filter, radii = render_pkg["render"], render_pkg["viewspace_points"], render_pkg["visibility_filter"], render_pkg["radii"]

        # Loss

        views = ["front view", "left side view", "back view", "right side view"]
        qangle = floor(4 * rand_angle / (2 * 3.14159))
        view = views[qangle]

        prompt = prompt.format(view)
        #text = tokenizer(["a photo of a beautiful twink statue, not noisy, not rainbow, cute, accurate anatomy, black background, greeco-roman statue"]).cuda()
        #text = tokenizer([f"a photo of a beautiful twink statue, {view}, not noisy, not rainbow, cute, accurate anatomy, black background, greeco-roman statue"]).cuda()
        #text = tokenizer(["a 3d model of a cyber-sigilist statue, sumerian god, celestialpunk, beautiful, monochrome shiny, gloss"]).cuda()
        text = tokenizer([prompt]).cuda()
        #text = tokenizer([f"a 3d model of an astronaut riding a horse, 3d asset, high quality, not noisy, beautiful, black background"]).cuda()
        #text = tokenizer(["a 3d model of the Mona Lisa, 3d asset, high quality, not noisy, beautiful, black background"]).cuda()
        #text = tokenizer([f"a 3d model of a tabby cat, {view}, 3d asset, high quality, not noisy, beautiful, black background"]).cuda()
        text_vec = model.encode_text(text).float()
        text_vec = text_vec / text_vec.norm(dim=-1, keepdim=True)

        crops = random_crops(image.unsqueeze(0), 32, min_size=0.7, max_size=0.9)
        img_vec = model.encode_image(clippp(crops).half()).float()
        img_vec = img_vec / img_vec.norm(dim=-1, keepdim=True)

        sim = img_vec @ text_vec.T
        clip_loss = -sim.mean()
        # self_sim = -(img_vec @ img_vec.T).mean()
        # loss = (1.0 - opt.lambda_dssim) * Ll1 + opt.lambda_dssim * (1.0 - ssim(image, gt_image))
        loss = clip_loss
        loss.backward()

        iter_end.record()

        if ((iteration % 10) == 0 or iteration == opt.iterations) and iteration > 700:
            rand_pos = torch.tensor([0.0, 0.0, camera_extent])
            angle = 3.14159 * 2.0 * iteration / opt.iterations
            rand_pos = camera_extent * torch.tensor([torch.tensor(angle).cos(), 0.0, torch.tensor(angle).sin()])

            rot, trans = look_at(rand_pos, torch.tensor([0.0, 0.0, 0.0]), torch.tensor([0.0, 1.0, 0.0]))

            dbg_cam = Camera(
                colmap_id=0,
                R=rot.numpy(), #rand_angle.numpy(),
                T=trans.numpy(), #torch.zeros((3,)).numpy(),
                FoVx=90.0,
                FoVy=90.0,
                image=torch.zeros((3, 512, 512)),
                gt_alpha_mask=torch.ones((512, 512)),
                image_name="test",
                uid=0,
            )
            image = render(dbg_cam, gaussians, pipe, background)["render"]

            show(image)
            camera.snap()

        if iteration == opt.iterations:
            return camera.animate(blit = False, interval = 50), gaussians


        with torch.no_grad():
            # Progress bar
            ema_loss_for_log = 0.4 * loss.item() + 0.6 * ema_loss_for_log
            if iteration % 10 == 0:
                progress_bar.set_postfix({"Loss": f"{ema_loss_for_log:.{7}f}"})
                progress_bar.update(10)
            if iteration == opt.iterations:
                progress_bar.close()

            # Log and save
            # training_report(tb_writer, iteration, Ll1, loss, l1_loss, iter_start.elapsed_time(iter_end), testing_iterations, scene, render, (pipe, background))
            if (iteration in saving_iterations):
                print("\n[ITER {}] Saving Gaussians".format(iteration))
                scene.save(iteration)

            # Densification
            if iteration < opt.densify_until_iter:
                # Keep track of max radii in image-space for pruning
                gaussians.max_radii2D[visibility_filter] = torch.max(gaussians.max_radii2D[visibility_filter], radii[visibility_filter])

                gaussians.add_densification_stats(viewspace_point_tensor, visibility_filter)

                densify = iteration % opt.densification_interval == 0
                if iteration > opt.densify_from_iter and densify:
                    size_threshold = 20 if iteration > opt.opacity_reset_interval else None
                    gaussians.densify_and_prune(opt.densify_grad_threshold, 0.005, camera_extent, size_threshold)

                if iteration % opt.opacity_reset_interval == 0 or (dataset.white_background and iteration == opt.densify_from_iter):
                    gaussians.reset_opacity()

            # Optimizer step
            if iteration < opt.iterations and (iteration % 1) == 0:
                gaussians.optimizer.step()
                gaussians.optimizer.zero_grad(set_to_none = True)

            if (iteration in checkpoint_iterations):
                print("\n[ITER {}] Saving Checkpoint".format(iteration))
                torch.save((gaussians.capture(), iteration), scene.model_path + "/chkpnt" + str(iteration) + ".pth")

2025-06-25 02:10:30.048335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-25 02:10:36.453887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from tqdm import tqdm
try:
    tqdm._instances.clear()
except:
    pass

# Запуск обучения модели текстово-управляемого гауссовского сплаттинга


В этом блоке происходит:

- Задание текстового описания (промта) для генерации 3D-модели подсолнухов Ван Гога.
- Инициализация параметров обучения и оптимизации через парсер аргументов.
- Установка количества итераций и параметров скорости обучения.
- Фиксация случайного сидa для воспроизводимости.
- Запуск основной функции обучения `training` с заданными параметрами.
- Отображение полученной анимации результата в формате HTML5-видео.

Первый запуск может занять больше времени из-за компиляции (JIT) модели CLIP.

In [ ]:
from IPython.core.display import HTML

prompt = "a 3d model of Van Gogh's Sunflowers, 3d asset, high quality, not noisy, beautiful, black background"
#prompt = "a 3d model of a tabby cat, {}, 3d asset, high quality, not noisy, beautiful, black background"
# Put a {} in your prompt to have view depedent prompting
# i.e "a cat, {}" -> "a cat, front view", "a cat, side view" etc

parser = ArgumentParser()
model_params = ModelParams(parser)
opt_params = OptimizationParams(parser)
pp = PipelineParams(parser)
args, _ = parser.parse_known_args()
args.iterations = 2500
args.position_lr_init = 1e-2
args.position_lr_final = 1e-5
args.position_lr_max_steps = args.iterations
# args.opacity_reset_interval = 5000
torch.manual_seed(2023)
animation, gaussians = training(prompt, model_params.extract(args), opt_params.extract(args), pp.extract(args), [], [], [], None, -1)
HTML(animation.to_html5_video())

Number of points at initialisation :  100



Training progress:  55%|█████▌    | 1380/2500 [01:27<01:16, 14.60it/s, Loss=-0.4009576]WARNING:matplotlib.image:Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).

Training progress:  56%|█████▌    | 1390/2500 [01:27<01:17, 14.25it/s, Loss=-0.4106680]WARNING:matplotlib.image:Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).

Training progress:  56%|█████▌    | 1400/2500 [01:28<01:21, 13.52it/s, Loss=-0.4105310]WARNING:matplotlib.image:Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).

Training progress:  56%|█████▋    | 1410/2500 [01:29<01:23, 13.03it/s, Loss=-0.3933875]WARNING:matplotlib.image:Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).

Training progress:  57%|█████▋    | 1420/2500 [01:30<01:22, 13.04it/s, Loss=-0.3982099]WARNING:matplotlib.i

In [11]:
gaussians.save_ply("../outputTC.ply")